1- Se tiene un RDD con el registro de notas de los alumnos de la forma
(padrón, materia, nota, fecha). Se pide resolver utilizando PySpark:

a) Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.

In [139]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

strRdd = sc.textFile('/home/marcos/PycharmProjects/practicaSpark/Ej1.csv')
rdd = strRdd.map(lambda x: x.split(","))
rdd = rdd.map(lambda x: (pd.to_numeric(x[0], 'coerce'), x[1], pd.to_numeric(x[2], 'coerce'),
                         pd.to_datetime(x[3], 'coerce')))

rdd.cache()

passedExams = rdd.filter(lambda x: (x[2] >= 4) & (x[3].year >= 2018))
passedExams.map(lambda x: x[0]).distinct().count()

4

b) Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).


In [140]:
averageScorePerStudent = rdd.map(lambda x: (x[0], (x[2], 1))).reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))
averageScorePerStudent = averageScorePerStudent.map(lambda x: (x[0], x[1][0]/x[1][1]))
averageScorePerStudent.cache()
averageScorePerStudent.collect()

[(99862, 5.851851851851852),
 (102342, 4.785714285714286),
 (102323, 6.5),
 (100125, 4.421052631578948)]

c) El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse
un segundo RDD alumnos con registros (padron, nombre y apellido).

In [141]:
strRdd = sc.textFile('/home/marcos/PycharmProjects/practicaSpark/Ej1c.csv')
studentsNames = strRdd.map(lambda x: x.split(","))
studentsNames = studentsNames.map(lambda x: (pd.to_numeric(x[0], 'coerce'), x[1]))

bestStudentID = averageScorePerStudent.reduce(lambda x,y: x if (x[1] > y[1]) else y)
studentsNames.filter(lambda x: x[0] == bestStudentID[0]).collect()

[(102323, ' Julio Cortazar')]